#**Automated Plant Leaf Disease Detection (APLDD)**
Project Group 25 Members:
*   Fabin Joe: 1005889709
*   Kshitij Bhardwaj:
*   Rahul Chopra:
*   Vishal Radhakrishnan:

Neural Network to analyze images of a plant leaf to identify various common diseases. Our project is indented to be used in the **Agricultural Sector** to result in early detection of diseases.

#Some Coding Guidelines so our code seems uniform
Class Definaitions: class **HappyPerson**(): 

Function Definations: def **makePersonHappy**():

Variable Definations: **measure_happiness** = 0


# Data Cleaning

In [1]:
# Function to Resize a given image
# - Vishal

2


In [ ]:
# Function to organize imags into directory
# - Fabin

In [ ]:
# Function to Create the csv file
# - Fabin

In [ ]:
# Function to relable and fill csv file
# - Rahul

In [ ]:
# Function to cause a train val test split
# - Kshitij

In [ ]:
# Function to segment images - optional maybe we can do this later.

# Baseline Model

# Neural Network